# Internal Residual Binning Logic for ForecasterEquivalentDate
This notebook demonstrates the internal residual binning logic of `ForecasterEquivalentDate` for safety-critical MLOps validation. We will inspect how residuals are computed and binned by predicted values.

In [ ]:
import numpy as np
import pandas as pd
from spotforecast2_safe.forecaster.recursive import ForecasterEquivalentDate
from spotforecast2_safe import __version__

print(f"Package version: {__version__}")

## Synthetic Time Series Generation
We create a periodic time series of 28 days (4 weeks) with a daily pattern to ensure that the 7-day offset produces consistent residuals.

In [ ]:
# Create 4 weeks of daily data
np.random.seed(42)
index = pd.date_range(start='2024-01-01', periods=28, freq='D')
# Sine wave (7-day period) + some noise
values = 10 + 5 * np.sin(2 * np.pi * np.arange(28) / 7) + np.random.normal(0, 0.5, 28)
y = pd.Series(values, index=index, name='synthetic_load')

y.plot(title="Synthetic Time Series (4 Weeks)", figsize=(10, 4))

## Forecaster Initialization
We initialize the forecaster with a 7-day offset and 3 bins for residuals to clearly see the quantile partitioning.

In [ ]:
# Initialize with 3 bins to facilitate inspection
forecaster = ForecasterEquivalentDate(
    offset=7, 
    binner_kwargs={'n_bins': 3, 'method': 'quantile'}
)

## Fit Method and Residual Binning
Calling `fit()` with `store_in_sample_residuals=True` triggers the internal call to `_binning_in_sample_residuals()`.

In [ ]:
# Fit the model
forecaster.fit(y=y, store_in_sample_residuals=True)

print(f"Model fitted: {forecaster.is_fitted}")

## Binner Intervals
The `QuantileBinner` partitions the predicted value space. Let's see how the 3 bins are defined.

In [ ]:
binner_intervals = forecaster.binner_intervals_
for bin_id, interval in binner_intervals.items():
    print(f"Bin {bin_id}: Range {interval}")

## Residual Distribution per Bin
Finally, we inspect `in_sample_residuals_by_bin_` to verify that residuals are correctly mapped and available for conformal prediction intervals.

In [ ]:
residuals_by_bin = forecaster.in_sample_residuals_by_bin_
for bin_id, residuals in residuals_by_bin.items():
    print(f"Bin {bin_id}: {len(residuals)} residuals stored.")
    print(f"  Sample residuals: {residuals[:5] if len(residuals) > 0 else 'None'}")